<a href="https://colab.research.google.com/github/laurindodumba/DATALAKE-AWS/blob/main/O1_EXTRA%C3%87%C3%83O_DE_DADOS_PROJETOCONSTRU%C3%87%C3%83O_DE_DATALAKE_AWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!mkdir -p data

In [26]:
# Importando as bibliotecas

import urllib.request


# 01 - EXTRAÇÃO DA BASE DE DADOS

In [27]:
import requests
import pandas as pd

# Endpoint OData para Pix liquidados por data
url = "https://olinda.bcb.gov.br/olinda/servico/SPI/versao/v1/odata/PixLiquidadosAtual?$format=json"

response = requests.get(url)
response.raise_for_status()   # garante que não deu erro

data = response.json()

# Normaliza o JSON (estrutura OData possui 'value' como lista de registros)
records = data.get("value", [])

df = pd.DataFrame(records)
print(df.head())

# Salva CSV localmente
df.to_csv("data/pix_liquidados_atual.csv", index=False)


         Data  Quantidade  CanalPrimario  CanalSecundario        Total   Media
0  2023-12-02   139072624    138976370.0          96254.0  23360889.99  167.98
1  2024-04-19   163588000    163124269.0         463731.0  79114284.18  483.62
2  2024-05-24   156097661    155912971.0         184690.0  68660685.32  439.86
3  2024-02-10   140720681    140509596.0         211085.0  25506025.17  181.25
4  2022-04-06    59981242            NaN              NaN  32855707.99  547.77


# 02 - CONECTANDO A COONRA DE ARMAZENAMENTO DA AWS

In [ ]:
!pip install boto3

In [32]:
import boto3


aws_access_key_id = "Insere a Chave"
aws_secret_access_key = "Insere a Chave"
region_name = "sa-east-1"

boto3.setup_default_session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name,
)

# Criação de um cliente S3
s3 = boto3.client("s3")

In [33]:
content = """
 Olá, estou estudando no dia para AWS

 """
with open("teste.txt", "w+") as f:
  f.write(content)

In [37]:
s3.upload_file("teste.txt", "datalake-pipeline01", "bronze/teste")

In [42]:
# Enviar os dados para o AWS - S3
from io import BytesIO
import time
import pandas as pd

# Initializar o Buffer
parquet_buffer = BytesIO()

# Transformação para Parquet
df.to_parquet(parquet_buffer, index=False)

# Extração da Data
df['Data'] = pd.to_datetime(df['Data'])
ano = df['Data'].dt.year.max()

s3.put_object(
        Bucket="datalake-pipeline01",
        Key=f"bronze/dados_pix_liquidados_{ano}.parquet",
        Body=parquet_buffer.getvalue(),
    )

{'ResponseMetadata': {'RequestId': 'YBM6XAYWM9J77944',
  'HostId': 'n866NDK5MN6IDYXyPL7ZdJvIQVUfEUn/nivhERAIoN22QGFqtlVfq5erKUfF1BK3EHG1Lp9y1v3/R3R7wE2s2Kucc3lQCADN',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'n866NDK5MN6IDYXyPL7ZdJvIQVUfEUn/nivhERAIoN22QGFqtlVfq5erKUfF1BK3EHG1Lp9y1v3/R3R7wE2s2Kucc3lQCADN',
   'x-amz-request-id': 'YBM6XAYWM9J77944',
   'date': 'Wed, 31 Dec 2025 22:22:27 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"444447bd4128c2fee41c86927f6ba6e6"',
   'x-amz-checksum-crc32': 'Ief4VA==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"444447bd4128c2fee41c86927f6ba6e6"',
 'ChecksumCRC32': 'Ief4VA==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}